In [6]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import csv
import pandas as pd
def get_url(adress,year,month,day):
    #請求抓取網頁資料
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    url = adress + "{0}-{1}-{2}".format(year,month,day)
    headers={'User-Agent':user_agent,} 

    request=urllib.request.Request(url,None,headers)
    response = urllib.request.urlopen(request)
    webpage = response.read()
    #---------------
    #soup為儲存網頁內容的變數
    soup = BeautifulSoup(webpage,'html.parser')
    # print(soup.prettify(), 'html.parser')
    # print(soup.p)
    return soup

def get_data(soup):
    data = soup.find_all("td")
    temp = []
    for i in range(10,len(data)):
        if data[i].text.strip() != "..." :
            temp.append(data[i].text.strip())
        else:
            temp.append(-1)
    temp = np.array(temp)
    temp = temp.reshape((len(temp) // 17,17))
    time = np.array( [data[4].text.strip()[5:]] *  temp.shape[0]  ).reshape((temp.shape[0] ,1))
    temp = np.hstack((time,temp))
    return  temp


In [7]:
def run(url,year,name):
    result = [[0] * 18]
    result[0][0] = "日期"
    result[0][1] = "觀測時間"
    result[0][2] = "測站氣壓"
    result[0][3] = '海平面氣壓'
    result[0][4] = '氣溫'
    result[0][5] = '露點溫度'
    result[0][6] = '相對溼度'
    result[0][7] = '風速'
    result[0][8] = '風向'
    result[0][9] = '最大陣風'
    result[0][10] = '最大陣風風向'
    result[0][11] = '降水量'
    result[0][12] = '降水時數'
    result[0][13] = '日照時數'
    result[0][14] = '全天空日射量'
    result[0][15] = '能見度'
    result[0][16] = '紫外線指數'
    result[0][17] = '總雲量'
    result = np.array(result)
    for i in range(1,13):
        month = []
        if year != 2016 :
            month= ['0','31','28','31','30','31','30','31','31','30','31','30','31']
        else:
            month= ['0','31','29','31','30','31','30','31','31','30','31','30','31']
        for j in range(1,int(month[i]) + 1):
            m = ""
            day = ""
            if i < 10 :
                m = "0" + str(i)
            else:
                m = str(i)
            
            if j < 10 :
                day = "0" + str(j)
            else:
                day = str(j) 
            soup = get_url(url,year,m,day)
            data = get_data(soup)
            result =  np.vstack((result,data))       

    with open('南投/%s%s.csv' %( year,name ), 'w', newline='') as csvfile:
      # 建立 CSV 檔寫入器
        writer = csv.writer(csvfile)
        writer.writerows(result)     
        print('%s年 %s站 finish' % (year , name))

In [10]:
%%time
data = pd.read_csv("埔里.csv")

for i in range(data.shape[0]):
    url = data.iloc[i][0]
    name = data.iloc[i][1]
    for j in range(2018,2020):
        run(url,j,name)

2018年 埔里站 finish
2019年 埔里站 finish
Wall time: 7min 7s
